In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [7]:
try:
    # Attempt to load the CSV file with ISO-8859-1 encoding
    df = pd.read_csv('finaldata.csv', encoding='ISO-8859-1')
    print(df.head())  # Display the first few rows to check the content
except Exception as e:
    print(f"An error occurred: {e}")


   No   _address                                              _body  \
0   1  AX-FEDBNK  Rs 40.00 debited from your A/c using UPI on 01...   
1   2  AX-FEDBNK  Rs 50.00 debited from your A/c using UPI on 01...   
2   3  AD-FEDBNK  Rs 75.00 debited from your A/c using UPI on 01...   
3   4  AD-FEDBNK  Rs 200.00 debited from your A/c using UPI on 0...   
4   5  AX-FEDBNK  Rs 255.00 debited from your A/c using UPI on 0...   

               date  
0  01-08-2023 08:51  
1  01-08-2023 11:56  
2  01-08-2023 12:05  
3  01-08-2023 17:09  
4  02-08-2023 16:25  


In [9]:
import re
# Define a function to extract amount and action from the message
def extract_details(text):
    # Regex to find the amount pattern and the action "debited"
    amount_match = re.search(r'Rs (\d+\.?\d*)', text)
    action_match = re.search(r'(debited|credited)', text, re.IGNORECASE)

    amount = amount_match.group(1) if amount_match else None
    action = action_match.group(1).lower() if action_match else None
    return pd.Series([amount, action])

# Apply the function to the '_body' column and create new columns
df[['Amount', 'Action']] = df['_body'].apply(extract_details)

# Show the modified DataFrame
print(df[['No', '_address', '_body', 'date', 'Amount', 'Action']].head())


   No   _address                                              _body  \
0   1  AX-FEDBNK  Rs 40.00 debited from your A/c using UPI on 01...   
1   2  AX-FEDBNK  Rs 50.00 debited from your A/c using UPI on 01...   
2   3  AD-FEDBNK  Rs 75.00 debited from your A/c using UPI on 01...   
3   4  AD-FEDBNK  Rs 200.00 debited from your A/c using UPI on 0...   
4   5  AX-FEDBNK  Rs 255.00 debited from your A/c using UPI on 0...   

               date  Amount   Action  
0  01-08-2023 08:51   40.00  debited  
1  01-08-2023 11:56   50.00  debited  
2  01-08-2023 12:05   75.00  debited  
3  01-08-2023 17:09  200.00  debited  
4  02-08-2023 16:25  255.00  debited  


In [12]:
# Create a new column 'Transaction_Type' based on the 'Action' column
df['Transaction_Type'] = df['Action'].apply(lambda x: 1 if x == 'credited' else 0)

# Show the modified DataFrame with the new binary column
print(df[['No', '_address', '_body', 'date', 'Amount', 'Action', 'Transaction_Type']].head())


   No   _address                                              _body  \
0   1  AX-FEDBNK  Rs 40.00 debited from your A/c using UPI on 01...   
1   2  AX-FEDBNK  Rs 50.00 debited from your A/c using UPI on 01...   
2   3  AD-FEDBNK  Rs 75.00 debited from your A/c using UPI on 01...   
3   4  AD-FEDBNK  Rs 200.00 debited from your A/c using UPI on 0...   
4   5  AX-FEDBNK  Rs 255.00 debited from your A/c using UPI on 0...   

               date  Amount   Action  Transaction_Type  
0  01-08-2023 08:51   40.00  debited                 0  
1  01-08-2023 11:56   50.00  debited                 0  
2  01-08-2023 12:05   75.00  debited                 0  
3  01-08-2023 17:09  200.00  debited                 0  
4  02-08-2023 16:25  255.00  debited                 0  


In [13]:
# Check for missing values in each column
print(df.isnull().sum())

# Handling missing values
# For numerical columns, fill missing values with the median or meandf['Amount'] = pd.to_numeric(df['Amount'], errors='coerce')
df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce')
df['Amount'] = df['Amount'].fillna(df['Amount'].median())

# For categorical columns, you might consider filling missing values with the mode or a placeholder like 'Unknown'
df['Action'] = df['Action'].fillna(df['Action'].mode()[0])

# If 'Action' has very few missing values, consider dropping those rows
df.dropna(subset=['Action'], inplace=True)


No                    0
_address              0
_body                 0
date                  0
Amount              594
Action              234
Transaction_Type      0
dtype: int64


In [14]:
# Assuming 'Action' is the only categorical variable that needs encoding
df['Action_Code'] = df['Action'].astype('category').cat.codes

In [15]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Normalization
scaler = MinMaxScaler()
df['Amount_Normalized'] = scaler.fit_transform(df[['Amount']])

# Standardization
scaler = StandardScaler()
df['Amount_Standardized'] = scaler.fit_transform(df[['Amount']])

In [17]:
# Review the DataFrame
print(df.head())

# Export the cleaned and preprocessed DataFrame
df.to_csv('finaldata_cleaned.csv', index=False)

   No   _address                                              _body  \
0   1  AX-FEDBNK  Rs 40.00 debited from your A/c using UPI on 01...   
1   2  AX-FEDBNK  Rs 50.00 debited from your A/c using UPI on 01...   
2   3  AD-FEDBNK  Rs 75.00 debited from your A/c using UPI on 01...   
3   4  AD-FEDBNK  Rs 200.00 debited from your A/c using UPI on 0...   
4   5  AX-FEDBNK  Rs 255.00 debited from your A/c using UPI on 0...   

               date  Amount   Action  Transaction_Type  Action_Code  \
0  01-08-2023 08:51    40.0  debited                 0            1   
1  01-08-2023 11:56    50.0  debited                 0            1   
2  01-08-2023 12:05    75.0  debited                 0            1   
3  01-08-2023 17:09   200.0  debited                 0            1   
4  02-08-2023 16:25   255.0  debited                 0            1   

   Amount_Normalized  Amount_Standardized  
0           0.001444            -0.163380  
1           0.001815            -0.155692  
2           0.

In [19]:
import pandas as pd

# Load your cleaned data
df = pd.read_csv('finaldata_cleaned.csv', encoding='ISO-8859-1')

# Convert 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'], errors='coerce')  # Adjust format if necessary

# Check the conversion
print(df['date'].head())


0   2023-01-08 08:51:00
1   2023-01-08 11:56:00
2   2023-01-08 12:05:00
3   2023-01-08 17:09:00
4   2023-02-08 16:25:00
Name: date, dtype: datetime64[ns]


In [20]:
# Extract year and month from the date for grouping data later
df['Year'] = df['date'].dt.year
df['Month'] = df['date'].dt.month

In [21]:
# Group by year and month, then summarize spending
monthly_spend = df.groupby(['Year', 'Month'])['Amount'].agg(['sum', 'mean']).reset_index()
monthly_spend.columns = ['Year', 'Month', 'Total_Spend', 'Average_Spend']

print(monthly_spend.head())


     Year  Month  Total_Spend  Average_Spend
0  2023.0    1.0      1927.95     148.303846
1  2023.0    2.0       320.00      80.000000
2  2023.0    3.0      1574.00     131.166667
3  2023.0    4.0       262.00      52.400000
4  2023.0    5.0        45.00      45.000000


In [22]:
# Overall average transaction size
average_transaction_size = df['Amount'].mean()
print(f"Average Transaction Size: {average_transaction_size}")


Average Transaction Size: 252.51754468485422


In [23]:
# Count number of transactions per category
transactions_per_category = df['Action'].value_counts().reset_index()
transactions_per_category.columns = ['Action', 'Transaction_Count']

print(transactions_per_category)


     Action  Transaction_Count
0   debited                938
1  credited                125


In [25]:
# Merging monthly data back to the main DataFrame (if necessary)
df = df.merge(monthly_spend, on=['Year', 'Month'], how='left')

# Save the enhanced DataFrame
df.to_csv('finaldata_enhanced.csv', index=False)


In [27]:
# Fill missing values with the mean or median
df['Amount'].fillna(df['Amount'].median(), inplace=True)

# Remove duplicates
df.drop_duplicates(inplace=True)


C:\Users\heman\AppData\Local\Temp\ipykernel_18044\1297187390.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Amount'].fillna(df['Amount'].median(), inplace=True)


In [29]:
def categorize_by_amount(amount):
    if amount < 50:
        return 'Miscellaneous'
    elif 100 <= amount < 500:
        return 'Food/Fuel/Shopping'
    elif amount >= 500:
        return 'Rent/High-Value Purchases'
    else:
        return 'Other'

# Ensure the 'Amount' column is numeric
df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce')

# Apply the function to create a new column
df['Amount_Category'] = df['Amount'].apply(categorize_by_amount)

In [30]:
# Ensure the 'Amount' column is numeric
df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce')

# Apply the function to create a new column
df['Amount_Category'] = df['Amount'].apply(categorize_by_amount)

# Check the new categorization
print(df[['Amount', 'Amount_Category']].head())

   Amount     Amount_Category
0    40.0       Miscellaneous
1    50.0               Other
2    75.0               Other
3   200.0  Food/Fuel/Shopping
4   255.0  Food/Fuel/Shopping


In [31]:
# Count the number of transactions per category
category_counts = df['Amount_Category'].value_counts()
print(category_counts)

# Mean spending per category
mean_spending = df.groupby('Amount_Category')['Amount'].mean()
print(mean_spending)


Amount_Category
Other                        676
Miscellaneous                185
Food/Fuel/Shopping           134
Rent/High-Value Purchases     68
Name: count, dtype: int64
Amount_Category
Food/Fuel/Shopping            198.090000
Miscellaneous                  24.605135
Other                          69.656805
Rent/High-Value Purchases    2797.678529
Name: Amount, dtype: float64


In [ ]:
# Contact list retrieval
import pandas as pd

# Load transaction data
df_transactions = pd.read_csv('/mnt/data/finaldata_enhanced.csv', encoding='ISO-8859-1')

# Load contact list - replace 'path_to_contact_list.csv' with the actual path
df_contacts = pd.read_csv('path_to_contact_list.csv')

In [44]:
# Assuming the correct column name is 'PhoneNumber'
df_contact['PhoneNumber'] = df_contact['PhoneNumber'].str.replace(' ', '').str.replace('-', '').str.replace('(', '').str.replace(')', '')
df_contact['PhoneNumber'] = df_contact['PhoneNumber'].str.replace('+91', '')  # Removing country code if present

# Handling Missing Values
df_contacts.dropna(subset=['Name', 'PhoneNumber'], inplace=True)  # Adjust to the actual column name

# Removing Duplicates
df_contacts.drop_duplicates(subset=['PhoneNumber'], keep='first', inplace=True)  # Adjust to the actual column name

# Correcting Names to title case for uniformity
df_contacts['Name'] = df_contacts['Name'].str.title()

# Save the cleaned data
df_contacts.to_csv('cleaned_contacts.csv', index=False)

# Display the cleaned data
print(df_contacts.head())


KeyError: 'PhoneNumber'

In [58]:
#Contacts

import pandas as pd

# Load the contact file
df_contacts = pd.read_csv('contact.csv')

# Display the first few rows to understand the structure
print(df_contacts.head())

# Check for missing values in each column
print("\nMissing Values in Each Column:")
print(df_contacts.isnull().sum())

# Check for duplicate entries based on the 'Mobile Phone' column
print("\nDuplicate Entries Check:")
print(df_contacts.duplicated(subset=['Mobile Phone']).sum())

# Print basic statistics to understand the data better
print("\nData Summary:")
print(df_contacts.describe(include='all'))


import pandas as pd

# Load the data ensuring phone numbers are read as strings
df_contacts = pd.read_csv('C:\\Users\\heman\\Desktop\\SpendSense\\contact.csv', dtype={'Mobile Phone': str})

# Check for formatting issues and attempt to clean
df_contacts['Mobile Phone'] = df_contacts['Mobile Phone'].apply(lambda x: x.split('.')[0] if isinstance(x, str) and '.' in x else x)

# Optionally, remove rows with missing mobile phone numbers if the phone number is essential
df_contacts.dropna(subset=['Mobile Phone'], inplace=True)

# Fill missing names with a placeholder if needed
df_contacts['Name'].fillna('Unknown', inplace=True)

# Remove duplicates based on 'Mobile Phone' assuming it should be unique
df_contacts = df_contacts.drop_duplicates(subset=['Mobile Phone'], keep='first')

# Display the cleaned data
print(df_contacts.head())
print("\nData Summary:")
print(df_contacts.describe(include='all'))





               Name Mobile Phone
0  Samsung Helpline  1.80041E+12
1      Naresh Mamma  9.19308E+11
2  Harsh Gautam SVM  9.18605E+11
3           Dada Ji   9450139031
4           Nana Ji  9.19415E+11

Missing Values in Each Column:
Name              5
Mobile Phone    874
dtype: int64

Duplicate Entries Check:
999

Data Summary:
        Name Mobile Phone
count   1775          906
unique  1773          780
top     Kush        53432
freq       2            8
               Name Mobile Phone
0  Samsung Helpline            1
1      Naresh Mamma            9
3           Dada Ji   9450139031
5       Canera Bank   7312566641
6     Sachin Chacha   9935161683

Data Summary:
                  Name Mobile Phone
count              454          454
unique             453          454
top     AL hello tunes            1
freq                 2            1


C:\Users\heman\AppData\Local\Temp\ipykernel_18044\2698033285.py:36: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_contacts['Name'].fillna('Unknown', inplace=True)
